# Fine-tuning BERT on a question-answering task

In [ ]:
! pip install git+https://github.com/huggingface/datasets.git

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-i7826ud_
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-i7826ud_
     |████████████████████████████████| 212 kB 4.5 MB/s 
     |████████████████████████████████| 136 kB 20.5 MB/s 
     |████████████████████████████████| 1.1 MB 50.8 MB/s 
     |████████████████████████████████| 77 kB 2.5 MB/s 
     |████████████████████████████████| 127 kB 73.8 MB/s 
     |████████████████████████████████| 94 kB 375 kB/s 
     |████████████████████████████████| 271 kB 74.0 MB/s 
     |████████████████████████████████| 144 kB 74.9 MB/s 
  Created wheel for datasets: filename=datasets-2.1.1.dev0-py3-none-any.whl size=333947 sha256=e4a456ce131d8ad01f976dc1f78914a45e88e589407835ebbff7f6393f6744b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-0prbem29/wheels/cc/33/c1/2cacc415b23189a83908e45db67381ba26175ef1e8aa9062aa
Successfully built datasets
  Attempting uninstall: urllib3
    Fou

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 880 kB 63.6 MB/s 
     |████████████████████████████████| 596 kB 64.5 MB/s 
     |████████████████████████████████| 6.6 MB 46.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a3e445959d9e530bfd5acd0e34f108770dcd46cc6853674ce70fc54998ddc3ee
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers

print(transformers.__version__)

4.18.0


In [ ]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
import pickle as pkl

In [ ]:
with open('/content/drive/MyDrive/Spring22/CS769/Project/FInal/SQUAD_COQA_uniform.pkl' , 'rb') as f:
  datasets = pkl.load(f)

## Displaying a few random samples from train set

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset) , "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,title,context,question,answers,id,__index_level_0__
0,Genocide,"The first draft of the Convention included political killings, but these provisions were removed in a political and diplomatic compromise following objections from some countries, including the USSR, a permanent security council member. The USSR argued that the Convention's definition should follow the etymology of the term, and may have feared greater international scrutiny of its own Great Purge. Other nations feared that including political groups in the definition would invite international intervention in domestic politics. However leading genocide scholar William Schabas states: “Rigorous examination of the travaux fails to confirm a popular impression in the literature that the opposition to inclusion of political genocide was some Soviet machination. The Soviet views were also shared by a number of other States for whom it is difficult to establish any geographic or social common denominator: Lebanon, Sweden, Brazil, Peru, Venezuela, the Philippines, the Dominican Republic, Iran, Egypt, Belgium, and Uruguay. The exclusion of political groups was in fact originally promoted by a non-governmental organization, the World Jewish Congress, and it corresponded to Raphael Lemkin’s vision of the nature of the crime of genocide.”",Which distinguished academic of genocide highlighted several other countries opposed to including political genocide in the Convention?,"{'answer_start': [568], 'text': ['William Schabas']}",43971,126859
1,Buckingham_Palace,"Before Prince Albert's death, the palace was frequently the scene of musical entertainments, and the greatest contemporary musicians entertained at Buckingham Palace. The composer Felix Mendelssohn is known to have played there on three occasions. Johann Strauss II and his orchestra played there when in England. Strauss's ""Alice Polka"" was first performed at the palace in 1849 in honour of the queen's daughter, Princess Alice. Under Victoria, Buckingham Palace was frequently the scene of lavish costume balls, in addition to the usual royal ceremonies, investitures and presentations.",Which piece of music by Johann Strauss was first performed at Buckingham in honor of Princess Alice?,"{'answer_start': [325], 'text': ['Alice Polka']}",18446,52236
2,Mammal,"Mammals include the largest animals on the planet, the rorquals and other large whales, as well as some of the most intelligent, such as elephants, primates, including humans, and cetaceans. The basic body type is a four-legged land-borne animal, but some mammals are adapted for life at sea, in the air, in trees, or on two legs. The largest group of mammals, the placentals, have a placenta, which enables feeding the fetus during gestation. Mammals range in size from the 30–40 mm (1.2–1.6 in) bumblebee bat to the 33-meter (108 ft) blue whale.",Which mammal is the smallest?,"{'answer_start': [497], 'text': ['bumblebee bat']}",19460,55004
3,Richard_Feynman,"Following the completion of his PhD in 1942, Feynman held an appointment at the University of Wisconsin–Madison as an assistant professor of physics. The appointment was spent on leave for his involvement in the Manhattan project. In 1945, he received a letter from Dean Mark Ingraham of the College of Letters and Science requesting his return to UW to teach in the coming academic year. His appointment was not extended when he did not commit to return. In a talk given several years later at UW, Feynman quipped, ""It's great to be back at the only university that ever had the good sense to fire me.""",Where did Feynman get rejected for a job after getting his PhD?,"{'answer_start': [], 'text': []}",35313,101453
4,Bern,"The sessions of the City Parliament are public. Unlike members of the City Council, members of the City Parliament are not politicians by profession, and they are paid a fee based on their attendance. Any resident of Bern allowed to vote can be elected as a member of the City Parliament. The parliament

## Preprocessing the training data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

## Set useful parameters

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

## Remove unnecessary columns

In [ ]:
datasets['train'] = datasets['train'].remove_columns('__index_level_0__')

In [ ]:
datasets['train'].column_names

['title', 'context', 'question', 'answers', 'id']

## Parse the dataset through our preprocessing function

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/66 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Since our task is question answering, we use the `AutoModelForQuestionAnswering` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

## Define the training args

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

## Initialise a trainer to train the model with all the arguments and parameters

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/srmukundb/bert-base-uncased-finetuned-squad into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.6k/415M [00:00<?, ?B/s]

Download file runs/Apr08_07-47-14_eb4989d4147e/1649404271.1411054/events.out.tfevents.1649404271.eb4989d4147e.…

Download file runs/Apr07_07-13-21_231c495bf9fa/1649315620.15691/events.out.tfevents.1649315620.231c495bf9fa.73…

Download file runs/Apr08_07-47-14_eb4989d4147e/events.out.tfevents.1649404271.eb4989d4147e.71.0:  29%|##8     …

...

Clean file training_args.bin:  34%|###3      | 1.00k/2.98k [00:00<?, ?B/s]

Clean file runs/Apr07_07-13-21_231c495bf9fa/1649315620.15691/events.out.tfevents.1649315620.231c495bf9fa.73.1:…

Clean file runs/Apr08_07-47-14_eb4989d4147e/events.out.tfevents.1649404271.eb4989d4147e.71.0:   8%|8         |…

Clean file runs/Apr07_07-13-21_231c495bf9fa/events.out.tfevents.1649315620.231c495bf9fa.73.0:  11%|#         |…

Clean file pytorch_model.bin:   0%|          | 1.00k/415M [00:00<?, ?B/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 65808
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12339


Epoch,Training Loss,Validation Loss
1,1.128000,1.310435
2,0.728500,1.258628
3,0.511200,1.440289


Saving model checkpoint to bert-base-uncased-finetuned-squad/checkpoint-500
Configuration saved in bert-base-uncased-finetuned-squad/checkpoint-500/config.json
Model weights saved in bert-base-uncased-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-squad/checkpoint-500/special_tokens_map.json
tokenizer config file saved in bert-base-uncased-finetuned-squad/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-squad/special_tokens_map.json
Saving model checkpoint to bert-base-uncased-finetuned-squad/checkpoint-1000
Configuration saved in bert-base-uncased-finetuned-squad/checkpoint-1000/config.json
Model weights saved in bert-base-uncased-finetuned-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-squad/checkpoint-1000/tokenizer_config.json
Special tokens f

TrainOutput(global_step=12339, training_loss=0.9299936279870672, metrics={'train_runtime': 15445.1774, 'train_samples_per_second': 12.782, 'train_steps_per_second': 0.799, 'total_flos': 3.868968817638605e+16, 'train_loss': 0.9299936279870672, 'epoch': 3.0})

Since this training is particularly long, let's save the model just in case we need to restart.

## Save the model

In [ ]:
%cd /content/drive/MyDrive/Spring22/CS769/Project/FInal

/content/drive/MyDrive/Spring22/CS769/Project/FInal


In [ ]:
trainer.save_model("bert-squad-coqa-uniform")